In [3]:
import openai
import json
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
def get_current_weather(location, unit = "fahrenheit"):
    """
    Retreive the current temperature for a temperature
    """
    weather_info = {
        'location': location,
        'temperature': 70,
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }
    return json.dumps(weather_info)

In [6]:
completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
    functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
],
function_call="auto",
)

In [8]:
reply_content = completion.choices[0]
reply_content

<OpenAIObject at 0x7f7097eff410> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_current_weather",
      "arguments": "{\n  \"location\": \"Boston\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [23]:
function_name = reply_content.to_dict()['function_call']['name']
args = json.loads(reply_content.to_dict()['function_call']['arguments'])

In [57]:
print(function_name, '\n', args)

get_current_weather 
 {'location': 'Boston'}


In [47]:
(f"{function_name}({' , '.join(args[key] for key in args.keys())})")

'get_current_weather(Boston)'

In [52]:
# using repr to add quotes
output = f"{function_name}({', '.join(repr(args[key]) for key in args.keys())})"
output

"get_current_weather('Boston')"

In [53]:
eval(output)

'{"location": "Boston", "temperature": 70, "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'